In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression  


#Estou selecionado só 10mil para ir mais rápido
df_=pd.read_csv('/kaggle/input/exemplo-regresso-apostas/under.csv')[:10000]

todas_colunas=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']

#Nossa função fit recebe um um string tamanho 16 de 0s e 1s, e retorna a lucratividade da combinação
def somaLog(codigo_genetico):
    global df_, todas_colunas
    
    #Quais colunas que entraram na regressão
    #colunas='A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P'
    colunas=','.join([ todas_colunas[i] for i,e in enumerate([int(c) for c in codigo_genetico]) if e])
    
    #Fitra o df baseado nas colunas
    df=df_[(colunas+',PL').split(',')]


    SLs=[] 
    for i in range(20):
        #Embaralha o dataframe baseado no random_state i 
        df=df.sample(frac=1, random_state=i)

        #Divide em 2 mil linhas para teste e o restante treinamento
        df_test,df_train=df[:2000],df[2000:]

        #Os Xs são todas as colunas exceto a PL que será o Y
        X_train,Y_train = df_train.loc[:,(df_train.columns!='PL') ], df_train.PL
        X_test, Y_test  = df_test.loc[:,(df_test.columns!='PL') ], df_test.PL

        #Treina a regressão os dados de treinamento
        reg=LinearRegression().fit(X_train,Y_train)

        #Veifica a lucratividade nos dados de teste
        SLs+=[sum(np.log(1+y*y_pred) for y_pred,y in zip(reg.predict(X_test),Y_test) if y_pred>0 ) ]


    #Mostra a lucrativida média e colunas selecionadas que deram origem a essa lucratividade
    return np.mean(SLs)


#Exemplo de fit
somaLog('1111010011110001')

0.5522677269606644

In [2]:
#Configurações
TAM_POP=20  #Tamano da população (número para para não zuar o barraco, ok :)
N_BITS=len(todas_colunas)  #´Números de 0s e 1s do cromossomo
TAXA_DE_REPRODUCAO=0.95
TAXA_DE_MUTACAO=0.05

#Gera a população incial
pop=[]
for _ in range(TAM_POP):
    code=''.join([ str(np.random.randint(2)) for i in range(N_BITS)  ])
    pop+=[ {'code':code, 'fit':somaLog(code) } ]

pop

[{'code': '1011000011101001', 'fit': 0.561068223157016},
 {'code': '1110001111110010', 'fit': 0.6093599248503377},
 {'code': '1111010000100001', 'fit': 0.041601337737974455},
 {'code': '0011010101010110', 'fit': 0.43974534045924846},
 {'code': '0000011011010101', 'fit': 0.05123050541551351},
 {'code': '0111101100001101', 'fit': 0.17913548706804383},
 {'code': '1101110001110111', 'fit': 0.7181267738465656},
 {'code': '1001011111010111', 'fit': 0.3698809227441048},
 {'code': '1111001001101101', 'fit': 0.6255077713989342},
 {'code': '1111100100100010', 'fit': 0.1899958378459047},
 {'code': '1001111100100011', 'fit': 0.1863756488427012},
 {'code': '1101101010111100', 'fit': -0.1370670315413314},
 {'code': '1111001100110100', 'fit': -0.10020815866333939},
 {'code': '0111111101110110', 'fit': 0.749642138773591},
 {'code': '1111110001100011', 'fit': 0.8373659181322868},
 {'code': '1001010100010000', 'fit': -0.02765477052335117},
 {'code': '0001000100110001', 'fit': -0.03836335731241204},
 {'c

In [3]:
for n_gera in range(20):
    codes=[]
    
    #Para cada 2 indivudos gera novos 2 codigos genéticos a através do cruzamentos dada taxa de reprodução
    for pai,mae in zip(pop[:TAM_POP//2],pop[TAM_POP//2:]):
        corte=1+np.random.randint(N_BITS-1)
        if np.random.random()<TAXA_DE_REPRODUCAO:
            codes+=[pai['code'][:corte]+mae['code'][corte:corte+(N_BITS//2)]+pai['code'][corte+(N_BITS//2):],
                    mae['code'][:corte]+pai['code'][corte:corte+(N_BITS//2)]+mae['code'][corte+(N_BITS//2):] ]
        else:
            codes+=[pai['code'], mae['code']]

    #Para cada codigo genetico, toma bit a bit, troca os 0s por 1s, ou vice e versa dada taxa de mutação
    codes=[''.join([str(int(not(int(bit)))) if np.random.random()<TAXA_DE_MUTACAO  else bit for bit in code]) for code in codes ]


    #Adiciona os individuos a população, nesse momento pop tem 2*TAM_POP individuos
    pop+=[ {'code':code, 'fit':somaLog(code)} for code in codes]


    #Seleção por metodos do torneio para que pop tenha exatos TAM_POP individuos 
    pop=[ind1 if ind1['fit']>ind2['fit'] else ind2 for ind1,ind2 in zip(pop[:TAM_POP],pop[TAM_POP:])]

    #Embaralha os individuos da população
    np.random.shuffle(pop)

    print('Gera#:',n_gera,'Melhor fit:', max([ ind['fit'] for ind in pop ] )  )


print('Melhor combinação de colunas:', ','.join([ todas_colunas[i] for i,e in enumerate([int(c) for c in sorted(pop, key=lambda x: -x['fit'])[0]['code']]) if e]))

Gera#: 0 Melhor fit: 0.8373659181322868
Gera#: 1 Melhor fit: 0.869595155729515
Gera#: 2 Melhor fit: 0.869595155729515
Gera#: 3 Melhor fit: 0.9674205513034322
Gera#: 4 Melhor fit: 0.9773952813453318
Gera#: 5 Melhor fit: 1.0572481781422411
Gera#: 6 Melhor fit: 1.0572481781422411
Gera#: 7 Melhor fit: 1.0572481781422411
Gera#: 8 Melhor fit: 1.0572481781422411
Gera#: 9 Melhor fit: 1.065513719126025
Gera#: 10 Melhor fit: 1.065513719126025
Gera#: 11 Melhor fit: 1.0735612387836004
Gera#: 12 Melhor fit: 1.0857059911288842
Gera#: 13 Melhor fit: 1.0857059911288842
Gera#: 14 Melhor fit: 1.0857059911288842
Gera#: 15 Melhor fit: 1.0857059911288842
Gera#: 16 Melhor fit: 1.0857059911288842
Gera#: 17 Melhor fit: 1.0857059911288842
Gera#: 18 Melhor fit: 1.0857059911288842
Gera#: 19 Melhor fit: 1.0857059911288842
Melhor combinação de colunas: A,B,C,D,E,I,J,O
